In [1]:
# 1. Install all the dependencies in Colab environment i.e. Apache Spark 2.4.4 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz 
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [3]:
!pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [4]:
# 2. Setup Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [5]:
# 3. Start Spark Session
import findspark
findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
# download do http para arquivo local
!wget --quiet --show-progress https://raw.githubusercontent.com/niltontac/EspAnalise-EngDados/master/data/ad_action.csv
 
# carregar dados
df_spark = sc.read.csv("./ad_action.csv", inferSchema=True, header=False)
 
# ver algumas informações sobre os tipos de dados de cada coluna
df_spark.printSchema()

ad_action.csv.6     100%[===================>]  20.57M  64.0MB/s    in 0.3s    
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [10]:
from pyspark.sql.functions import to_timestamp

df_spark = df_spark.withColumnRenamed('_c0', 'timestamp')
df_spark = df_spark.withColumnRenamed('_c1', 'user_id')
df_spark = df_spark.withColumnRenamed('_c2', 'action')
df_spark = df_spark.withColumnRenamed('_c3', 'adId')
df_spark = df_spark.withColumnRenamed('_c4', 'campaignId')
df_spark = df_spark.withColumn('timestamp', to_timestamp('timestamp'))
df_spark.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- action: string (nullable = true)
 |-- adId: string (nullable = true)
 |-- campaignId: string (nullable = true)



In [11]:
df_spark.show(5)

+-------------------+--------------------+------+-------+-------------+
|          timestamp|             user_id|action|   adId|   campaignId|
+-------------------+--------------------+------+-------+-------------+
|2016-09-21 22:11:00|7c74953c-66cc-48b...| click|adId_09|campaignId_01|
|2016-06-25 18:29:00|676a083e-2f8e-4ff...|  view|adId_09|campaignId_02|
|2016-02-14 19:03:00|77158997-0dfa-48b...| click|adId_02|campaignId_02|
|2016-03-26 06:27:00|78aa2467-b502-413...| click|adId_07|campaignId_03|
|2016-01-02 04:57:00|fef9a98c-d73e-48e...|  view|adId_02|campaignId_02|
+-------------------+--------------------+------+-------+-------------+
only showing top 5 rows

